In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/rc-predict/rc_predict1.h5
/kaggle/input/ventilator-pressure-prediction/sample_submission.csv
/kaggle/input/ventilator-pressure-prediction/train.csv
/kaggle/input/ventilator-pressure-prediction/test.csv


In [2]:
##Predict R and C using RNN
##Analyze the ventillator pressure dataset
import pandas as pd
import numpy as np
##Create a NN model
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Dropout, LSTM, Bidirectional, GRU
from tensorflow.keras.callbacks import CSVLogger, TensorBoard, ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.preprocessing import timeseries_dataset_from_array
import gc


trainDataPath = "/kaggle/input/ventilator-pressure-prediction/train.csv"
df_data = pd.read_csv(trainDataPath, header=0)
df_data_orig = df_data.copy()
df_data.head()
#df_data.describe()

# testDataPath = "/kaggle/input/ventilator-pressure-prediction/test.csv"
# df_test = pd.read_csv(testDataPath, header=0)

#df_test.head()

DEBUG= False
if DEBUG:
    df_data = df_data.iloc[:80*100, :]
    #df_test = df_test.iloc[:80*100, :]
print(df_data.shape)
#print(df_test.shape)

(6036000, 8)


In [3]:
##Crreate targets based on R and C
# df_data['R'] = df_data['R'].astype(str)
# df_data['C'] = df_data['C'].astype(str)
df_data['R__C'] = df_data["R"].astype(str) + '__' + df_data["C"].astype(str)
df_data = pd.get_dummies(df_data)
df_data.head()


,id,breath_id,R,C,time_step,u_in,u_out,pressure,R__C_20__10,R__C_20__20,R__C_20__50,R__C_50__10,R__C_50__20,R__C_50__50,R__C_5__10,R__C_5__20,R__C_5__50
0,1,1,20,50,0.000000,0.083334,0,5.837492,0,0,1,0,0,0,0,0,0
1,2,1,20,50,0.033652,18.383041,0,5.907794,0,0,1,0,0,0,0,0,0
2,3,1,20,50,0.067514,22.509278,0,7.876254,0,0,1,0,0,0,0,0,0
3,4,1,20,50,0.101542,22.808822,0,11.742872,0,0,1,0,0,0,0,0,0
4,5,1,20,50,0.135756,25.355850,0,12.234987,0,0,1,0,0,0,0,0,0


In [4]:
##Get slices every 80steps. 
## https://stackoverflow.com/questions/25055712/pandas-every-nth-row
y_train = df_data.iloc[::80,:].iloc[:,8:].values
y_train = y_train.reshape(-1,1,9)
y_train.shape

(75450, 1, 9)

In [5]:
##Get x data in timeseries format
x_train = df_data["u_in"].values.reshape(-1,1,80) ##(batches, time_steps, features)
x_train.shape

(75450, 1, 80)

In [6]:
##Create RNN for training
from tensorflow.keras.layers import Dense, Activation, Dropout, LSTM, Bidirectional, GRU
tf.keras.backend.clear_session()

model = Sequential()
model.add(Bidirectional(GRU(1024,return_sequences=True), input_shape=x_train.shape[-2:]))
model.add(Bidirectional(GRU(512, return_sequences=True)))
# model.add(Bidirectional(GRU(512, return_sequences=True)))
# model.add(Bidirectional(GRU(256, return_sequences=True)))
model.add(Dense(128,activation='selu'))
model.add(Dense(9, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

model.summary()


User settings:

   KMP_AFFINITY=granularity=fine,verbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_SETTINGS=1
   KMP_WARNINGS=0

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=false
   KMP_ENABLE_TASK_THROTTLING=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_NUM_LOCKS_IN_BLOCK=1
   KMP_PLAIN_BARRIER='2,2'
   KMP_PLAIN_BARRIER_PATTERN='hyper,hyper'
   KMP_REDUCTION_BARRIER='1,1'
  

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 1, 2048)           6795264   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1, 1024)           7870464   
_________________________________________________________________
dense (Dense)                (None, 1, 128)            131200    
_________________________________________________________________
dense_1 (Dense)              (None, 1, 9)              1161      
Total params: 14,798,089
Trainable params: 14,798,089
Non-trainable params: 0
_________________________________________________________________


In [7]:
earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True,\
                                             verbose=1)

##Train the model
#history = model.fit(x=x_train, y=y_train, batch_size=32, epochs=10, validation_split=0.2,callbacks=[earlystop])
history = model.fit(x=x_train, y=y_train, batch_size=32, epochs=150, validation_split=0.2)

Epoch 1/150


2021-10-30 07:30:59.974077: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


1887/1887 [==============================] - 145s 73ms/step - loss: 0.9616 - categorical_accuracy: 0.6056 - val_loss: 0.7873 - val_categorical_accuracy: 0.6673
Epoch 2/150
1887/1887 [==============================] - 135s 72ms/step - loss: 0.6864 - categorical_accuracy: 0.7069 - val_loss: 0.6341 - val_categorical_accuracy: 0.7286
Epoch 3/150
1887/1887 [==============================] - 134s 71ms/step - loss: 0.5845 - categorical_accuracy: 0.7511 - val_loss: 0.5406 - val_categorical_accuracy: 0.7641
Epoch 4/150
1887/1887 [==============================] - 135s 72ms/step - loss: 0.5216 - categorical_accuracy: 0.7796 - val_loss: 0.5218 - val_categorical_accuracy: 0.7770
Epoch 5/150
1887/1887 [==============================] - 136s 72ms/step - loss: 0.4721 - categorical_accuracy: 0.8019 - val_loss: 0.4917 - val_categorical_accuracy: 0.7851
Epoch 6/150
1887/1887 [==============================] - 137s 73ms/step - loss: 0.4319 - categorical_accuracy: 0.8207 - val_loss: 0.4374 - val_categoric

In [8]:
model.save("./rc_predict1.h5")

In [9]:
# ##Load model
# #tf.keras.backend.clear_session()
# model1 = load_model("./rc_predict1.h5")
# model1.summary()

In [10]:
# predict = model.predict(x_train, batch_size=10240)
# predict.shape

In [11]:
# predict_new = predict.reshape(-1,9)
# y_pred = np.argmax(predict_new, axis=1)

In [12]:
# y_actual = np.argmax(y_train.reshape(-1,9), axis=1)